Course Human-Centered Data Science ([HCDS](https://www.mi.fu-berlin.de/en/inf/groups/hcc/teaching/winter_term_2020_21/course_human_centered_data_science.html)) - Winter Term 2020/21 - [HCC](https://www.mi.fu-berlin.de/en/inf/groups/hcc/index.html) | [Freie Universität Berlin](https://www.fu-berlin.de/)

***

# A2 - Wikipedia, ORES, and Bias in Data
Please follow the reproducability workflow as practiced during the last exercise.

## Step 1⃣ | Data acquisition

You will use two data sources: (1) Wikipedia articles of politicians and (2) world population data.

**Wikipedia articles -**
The Wikipedia articles can be found on [Figshare](https://figshare.com/articles/Untitled_Item/5513449). It contains politiciaans by country from the English-language wikipedia. Please read through the documentation for this repository, then download and unzip it to extract the data file, which is called `page_data.csv`.

**Population data -**
The population data is available in `CSV` format in the `_data` folder. The file is named `export_2019.csv`. This dataset is drawn from the [world population datasheet](https://www.prb.org/international/indicator/population/table/) published by the Population Reference Bureau (downloaded 2020-11-13 10:14 AM). I have edited the dataset to make it easier to use in this assignment. The population per country is given in millions!

In [99]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

### Read in the two raw data files

In [2]:
page_data = pd.read_csv("../data_raw/page_data.csv", sep=",")
page_data.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


In [3]:
# Country prefixes are seperated by commas, so the csv seperator has to be explicitly defined as semicolon 
population_data = pd.read_csv("../data_raw/export_2019.csv", sep=";")
population_data

,country,population,region
0,Algeria,44.357,AFRICA
1,Egypt,100.803,AFRICA
2,Libya,6.891,AFRICA
3,Morocco,35.952,AFRICA
4,Sudan,43.849,AFRICA
...,...,...,...
205,Samoa,0.200,OCEANIA
206,Solomon Islands,715.000,OCEANIA
207,Tonga,99.000,OCEANIA
208,Tuvalu,0.010,OCEANIA


## Step 2⃣ | Data processing and cleaning
The data in `page_data.csv` contain some rows that you will need to filter out. It contains some page names that start with the string `"Template:"`. These pages are not Wikipedia articles, and should not be included in your analysis. The data in `export_2019.csv` does not need any cleaning.

***

| | `page_data.csv` | | |
|-|------|---------|--------|
| | **page** | **country** | **rev_id** |
|0|	Template:ZambiaProvincialMinisters | Zambia | 235107991 |
|1|	Bir I of Kanem | Chad | 355319463 |

***

| | `export_2019.csv` | | |
|-|------|---------|--------|
| | **country** | **population** | **region** |
|0|	Algeria | 44.357 | AFRICA |
|1|	Egypt | 100.803 | 355319463 |

***

### Drop all rows containing a template page

In [4]:
page_data = page_data[~page_data.page.str.contains("Template")]
page_data.head()

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568


### Getting article quality predictions with ORES

Now you need to get the predicted quality scores for each article in the Wikipedia dataset. We're using a machine learning system called [**ORES**](https://www.mediawiki.org/wiki/ORES) ("Objective Revision Evaluation Service"). ORES estimates the quality of an article (at a particular point in time), and assigns a series of probabilities that the article is in one of the six quality categories. The options are, from best to worst:

| ID | Quality Category |  Explanation |
|----|------------------|----------|
| 1 | FA    | Featured article |
| 2 | GA    | Good article |
| 3 | B     | B-class article |
| 4 | C     | C-class article |
| 5 | Start | Start-class article |
| 6 | Stub  | Stub-class article |

For context, these quality classes are a sub-set of quality assessment categories developed by Wikipedia editors. If you're curious, you can [read more](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment#Grades) about what these assessment classes mean on English Wikipedia. For this assignment, you only need to know that these categories exist, and that ORES will assign one of these six categories to any `rev_id`. You need to extract all `rev_id`s in the `page_data.csv` file and use the ORES API to get the predicted quality score for that specific article revision.

### ORES REST API endpoint

The [ORES REST API](https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model) is configured fairly similarly to the pageviews API we used for the last assignment. It expects the following parameters:

* **project** --> `enwiki`
* **revid** --> e.g. `235107991` or multiple ids e.g.: `235107991|355319463` (batch)
* **model** --> `wp10` - The name of a model to use when scoring.

**❗Note on batch processing:** Please read the documentation about [API usage](https://www.mediawiki.org/wiki/ORES#API_usage) if you want to query a large number of revisions (batches). 

You will notice that ORES returns a prediction value that contains the name of one category (e.g. `Start`), as well as probability values for each of the six quality categories. For this assignment, you only need to capture and use the value for prediction.

**❗Note:** It's possible that you will be unable to get a score for a particular article. If that happens, make sure to maintain a log of articles for which you were not able to retrieve an ORES score. This log should be saved as a separate file named `ORES_no_scores.csv` and should include the `page`, `country`, and `rev_id` (just as in `page_data.csv`).

You can use the following **samle code for API calls**:

In [ ]:
import requests
import json

# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/YOUR-USER-NAME',
    'From': 'YOUR-EMAIL@fu-berlin.de'
}

def get_ores_data(rev_id, headers):
    
    # Define the endpoint
    # https://ores.wikimedia.org/scores/enwiki/?models=wp10&revids=807420979|807422778
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

    params = {'project' : 'enwiki',
              'model'   : 'wp10',
              'revids'  : rev_id
              }

    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    data = json.dumps(response)

    return data

### Define a function to query the ORES API

In [9]:
headers = {
    'User-Agent': 'https://github.com/jonas-weber',
    'From': 'webej51@fu-berlin.de'
}

def get_ores_data(rev_ids, headers):
    
    # Define the endpoint
    # https://ores.wikimedia.org/scores/enwiki/?models=wp10&revids=807420979|807422778
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

    params = {'project' : 'enwiki',
              'model'   : 'wp10',
              'revids'  : '|'.join([str(x) for x in rev_ids])
              }

    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()

    return response

Sending one request for each `rev_id` might take some time. If you want to send batches you can use `'|'.join(str(x) for x in revision_ids` to put your ids together. Please make sure to deal with [exception handling](https://www.w3schools.com/python/python_try_except.asp) of the `KeyError` exception, when extracting the `prediction` from the `JSON` response.

### Send requests for the ORES quality prediction in a given batch size, which must not be higher than 50, then store the results in a dictonary.
The dictonary is used before merging the page_data with the ores_prediction_data DataFrame because of performance benefits instead of editing the DataFrame after each API call.

In [22]:
rev_id_list = page_data.rev_id.tolist()
len(rev_id_list)

46701

In [24]:
batch_size = 50
ores_predictions = {}

for i in range(0, len(rev_id_list), batch_size):
    if i + batch_size > len(rev_id_list):
        ids = rev_id_list[i:]
    else:
        ids = rev_id_list[i:i+batch_size]
    predictions = get_ores_data(ids, headers)
    for x in predictions["enwiki"]["scores"]:
        try:
            ores_predictions[x] = predictions["enwiki"]["scores"][str(x)]["wp10"]["score"]["prediction"]
        except KeyError:
            ores_predictions[x] = np.nan

In [31]:
ores_prediction_data = pd.DataFrame(ores_predictions.items(), columns=["rev_id", "prediction"])
ores_prediction_data["rev_id"] = pd.to_numeric(ores_prediction_data["rev_id"])
ores_prediction_data.head()

,rev_id,prediction
0,355319463,Stub
1,393276188,Stub
2,393822005,Stub
3,395521877,Stub
4,395526568,Stub
...,...,...
46696,807482007,GA
46697,807483006,C
46698,807483153,GA
46699,807483270,C


### Merge the Dataframes to include the ORES quality prediction in the page_data Dataframe

In [41]:
page_data_full = page_data.merge(ores_prediction_data, on="rev_id")
page_data_full.head()

,page,country,rev_id,prediction
0,Bir I of Kanem,Chad,355319463,Stub
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,Stub
2,Yos Por,Cambodia,393822005,Stub
3,Julius Gregr,Czech Republic,395521877,Stub
4,Edvard Gregr,Czech Republic,395526568,Stub


### Write the Articles without a ORES prediction into the file "ORES_no_scores.csv" and drop them from the DataFrame

In [52]:
ores_no_scores = page_data_full[page_data_full["prediction"].isna()]
ores_no_scores = ores_no_scores.drop("prediction", axis=1)
ores_no_scores.to_csv("../data_clean/ORES_no_scores.csv", index=False)

page_data_full = page_data_full[~page_data_full["prediction"].isna()]

### Combining the datasets

Now you need to combine both dataset: (1) the wikipedia articles and its ORES quality scores and (2) the population data. Both have columns named `country`. After merging the data, you'll invariably run into entries which cannot be merged. Either the population dataset does not have an entry for the equivalent Wikipedia country, or vis versa.

Please remove any rows that do not have matching data, and output them to a `CSV` file called `countries-no_match.csv`. Consolidate the remaining data into a single `CSV` file called `politicians_by_country.csv`.

The schema for that file should look like the following table:


| article_name | country | region | revision_id | article_quality | population |
|--------------|---------|--------|-------------|-----------------|------------|
| Bir I of Kanem | Chad  | AFRICA | 807422778 | Stub | 16877000 |

In [64]:
combined_data = page_data_full.merge(population_data, on="country", how="outer")
merge_errors = combined_data[combined_data.isna().any(axis=1)]
merge_errors.to_csv("../data_clean/countries-no_match.csv", index=False)

combined_data = combined_data[~combined_data.isna().any(axis=1)]

combined_data.to_csv("../data_clean/politicians_by_country.csv", index=False)

## Step 3⃣ | Analysis

Your analysis will consist of calculating the proportion (as a percentage) of articles-per-population (we can also call it `coverage`) and high-quality articles (we can also call it `relative-quality`)for **each country** and for **each region**. By `"high quality"` arcticle we mean an article that ORES predicted as `FA` (featured article) or `GA` (good article).

**Examples:**

* if a country has a population of `10,000` people, and you found `10` articles about politicians from that country, then the percentage of `articles-per-population` would be `0.1%`.
* if a country has `10` articles about politicians, and `2` of them are `FA` or `GA` class articles, then the percentage of `high-quality-articles` would be `20%`.

### Results format

The results from this analysis are six `data tables`. Embed these tables in the Jupyter notebook. You do not need to graph or otherwise visualize the data for this assignment. The tables will show:

1. **Top 10 countries by coverage**<br>10 highest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Bottom 10 countries by coverage**<br>10 lowest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Top 10 countries by relative quality**<br>10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Bottom 10 countries by relative quality**<br>10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

**❗Hint:** You will find what country belongs to which region (e.g. `ASIA`) also in `export_2019.csv`. You need to calculate the total poulation per region. For that you could use `groupby` and also check out `apply`.

### Table 1

In [94]:
population_data = combined_data[["country", "population"]].drop_duplicates().reset_index(drop=True)
population = dict(zip(population_data["country"], population_data["population"]))
t1 = combined_data[["country","page"]].groupby(by="country", as_index=False).count()
t1["articles/population"] = t1.apply(lambda x: x["page"]/(population[x["country"]]*1000000), axis=1)
t1 = t1.drop(columns=["page"])
t1.nlargest(10, "articles/population").reset_index(drop=True)

,country,articles/population
0,Tuvalu,0.005400
1,Albania,0.000161
2,New Zealand,0.000157
3,Norway,0.000122
4,Moldova,0.000120
5,Estonia,0.000111
6,Finland,0.000103
7,Sao Tome and Principe,0.000100
8,Lithuania,0.000087
9,Uruguay,0.000081


### Table 2

In [97]:
t1.nsmallest(10, "articles/population").reset_index(drop=True)

,country,articles/population
0,Guyana,2.541296e-08
1,Djibouti,3.744939e-08
2,Belize,3.818616e-08
3,Barbados,4.878049e-08
4,Bahamas,5.089059e-08
5,Cape Verde,6.474820e-08
6,Suriname,6.611570e-08
7,French Guiana,9.183673e-08
8,Martinique,9.550562e-08
9,Montenegro,1.157556e-07


### Table 3

In [101]:
article_count = dict(Counter(combined_data["country"]))
qual_article_count = dict(Counter(combined_data.loc[combined_data["prediction"].isin(["GA", "FA"])]["country"]))
zero_qual_articles = combined_data.loc[~combined_data["country"].isin(qual_article_count.keys())]["country"].unique()
qual_article_count.update(dict(zip(zero_qual_articles, np.zeros(len(qual_article_count)))))

t3 = pd.DataFrame(combined_data["country"]).drop_duplicates()
t3["quality_article_rate"] = t3["country"].apply(lambda x: qual_article_count[x]/article_count[x])

t3.nlargest(10, "quality_article_rate").reset_index(drop=True)

,country,quality_article_rate
0,"Korea, North",0.222222
1,Saudi Arabia,0.128205
2,Romania,0.122449
3,Central African Republic,0.121212
4,Uzbekistan,0.107143
5,Mauritania,0.104167
6,Guatemala,0.084337
7,Dominica,0.083333
8,Syria,0.078125
9,Benin,0.076923


### Table 4

In [104]:
t3.nsmallest(10, "quality_article_rate").reset_index(drop=True)

,country,quality_article_rate
0,French Guiana,0.0
1,Angola,0.0
2,Finland,0.0
3,Tunisia,0.0
4,San Marino,0.0
5,Moldova,0.0
6,Monaco,0.0
7,Turkmenistan,0.0
8,Grenada,0.0
9,Tonga,0.0


### Table 5

In [110]:
region_article_count = dict(Counter(combined_data["region"]))
t5 = combined_data[["population", "region"]].groupby(by="region", as_index=False).sum()
t5["articles/population"] = t5.apply(lambda x: region_article_count[x["region"]]/x["population"] , axis=1)
t5 = t5.drop(columns=["population"]).sort_values(by="articles/population", ascending=False).reset_index(drop=True)
t5

,region,articles/population
0,EUROPE,0.018669
1,AFRICA,0.017800
2,LATIN AMERICA AND THE CARIBBEAN,0.012231
3,OCEANIA,0.009578
4,NORTHERN AMERICA,0.004964
5,ASIA,0.003171


### Table 6

In [112]:
region_qual_article_count = dict(Counter(combined_data.loc[combined_data["prediction"].isin(["GA", "FA"])]["region"]))

t6 = pd.DataFrame(combined_data["region"]).drop_duplicates()
t6["quality_article_rate"] = t6["region"].apply(lambda x: region_qual_article_count[x]/region_article_count[x])
t6 = t6.sort_values(by="quality_article_rate", ascending=False).reset_index(drop=True)
t6

,region,quality_article_rate
0,NORTHERN AMERICA,0.054450
1,ASIA,0.027029
2,EUROPE,0.022186
3,OCEANIA,0.020147
4,AFRICA,0.017387
5,LATIN AMERICA AND THE CARIBBEAN,0.014413


***

#### Credits

This exercise is slighty adapted from the course [Human Centered Data Science (Fall 2019)](https://wiki.communitydata.science/Human_Centered_Data_Science_(Fall_2019)) of [Univeristy of Washington](https://www.washington.edu/datasciencemasters/) by [Jonathan T. Morgan](https://wiki.communitydata.science/User:Jtmorgan).

Same as the original inventors, we release the notebooks under the [Creative Commons Attribution license (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/).